In [2]:
import torch
import torch.nn as nn

device = "mps" if torch.backends.mps.is_available() else "cpu"

In [2]:
'''
Basic arithmetic with broadcasting.
Create a tensor A of shape (3, 1) and a tensor B of shape (1, 4). 
Perform element-wise addition and element-wise multiplication on A and B. 
Explain how broadcasting allows this operation to be performed.
'''

A = torch.rand(size=(3,1)).to(device)
B = torch.rand(size=(1,4)).to(device)

n = A.shape[0]
m = B.shape[1]

C = torch.zeros(size=(n,m)).to(device)
for i in range(n):
    for j in range(m):
        C[i][j] = A[i][0] + B[0][j]

print(C)


tensor([[1.2390, 0.8470, 0.6567, 0.6765],
        [1.7383, 1.3463, 1.1559, 1.1758],
        [0.8531, 0.4611, 0.2707, 0.2906]], device='mps:0')


In [5]:
C_= torch.broadcast_shapes((3,1),(1,4)) # C gets broadcasted shape of (3,4)
C_ = A+B
assert(torch.sum(C_ == C) == C.shape[0]*C.shape[1]) 
# Number of equal elements in both the tensors should be equal to the tot number of tensors

### Broadcasting:
- In linear algebra, addition, subtraction, multiplication, and division of matrices (or tensors) require them to have the exact same shape (element-wise operations).
- Broadcasting is a mechanism that relaxes this constraint. When two tensors have different shapes, PyTorch (and NumPy) attempts to align them by "stretching" the smaller tensor along its dimension(s) so that the resulting shapes are compatible. 



### Rule of Broadcasting:
- Each tensor has at least one dimension.
- When iterating over the dimension sizes, starting at the trailing dimension, the dimension sizes must either be equal, one of them is 1, or one of them does not exist.
- If a dimension is missing, always pad in the leading dimensions. Eg. (4,) -> (1,4) 

Eg. A = (3,1,2), B = (2,)
These 2 are broadcastable because:

```
A = (3,1,2)
B = (1,1,2)
```

And hence the result of the broadcasted tensor is (3,1,2)

Eg. 

```
A = (4, 1, 6, 1)
B = (1, 5, 1, 8)
```

C = A + B <br>
C = (4,5,6,8)

In [16]:
# Write a function `is_broadcastable(shape1, shape2)` 
# that returns True/False if the two shapes can be broadcast together
# according to PyTorch rules.
#
# Example:
# is_broadcastable((4,1,3), (1,5,1)) ➜ True
# is_broadcastable((4,2), (3,)) ➜ False
'''
a = (2,3,4)
b = (3,4)
Ans = True
'''

def is_broadcastable(A: torch.Tensor, B:torch.tensor)-> bool:
    shape1 = A.shape
    shape2 = B.shape

    if len(shape1) > len(shape2):
        while(len(shape1) != len(shape2)):
            B = torch.unsqueeze(B, dim=0)
            shape2 = B.shape
    else:
        while(len(shape1) != len(shape2)):
            A = torch.unsqueeze(A, dim=0)
            shape1 = A.shape
    
    assert(len(shape1) == len(shape2))
    print(f"Shape of A : {A.shape} \nShape of B : {B.shape}")
    
    n = len(shape1)
    
    for i in range(n):
        if ((shape1[n-i-1] != shape2[n-i-1]) and (shape1[n-i-1] != 1) and (shape2[n-i-1] != 1)):
            return False,None,None
    
    return True,shape1, shape2


In [17]:
A = torch.rand(size=(2,3,4))
B = torch.rand(size=(3,4))
is_broadcastable(A,B)

Shape of A : torch.Size([2, 3, 4]) 
Shape of B : torch.Size([1, 3, 4])


(True, torch.Size([2, 3, 4]), torch.Size([1, 3, 4]))

In [45]:
# Create a tensor of shape (3,1)
# Expand it to (3,5) using .expand()
# Verify that .expand() doesn’t allocate new memory (use a.is_shared_storage(b))

a = torch.rand(size=(3,1)).to("mps")
b = torch.expand_copy(a,size=(3,5)).to("mps")
print(a.data_ptr())
print(b.data_ptr())

4631818272
5144374112


In [35]:
# Given x = torch.rand(3,1,5) and y = torch.rand(1,4,1)
# Perform x + y and print shape.
# Confirm with manual reasoning.
x = torch.rand(3,1,5)
y = torch.rand(1,4,1)

z = x+y
print(z.shape)
z_ = torch.broadcast_shapes(x.shape,y.shape)
# assert(z.shape == z_.shape)
assert(z.shape == z_)

torch.Size([3, 4, 5])


In [43]:
# Given a tensor X of shape (batch_size, num_features)
# Subtract mean of each feature and divide by its std, 
# using broadcasting (without explicit loops).
# X_norm = (X - X.mean(dim=0)) / X.std(dim=0)

batch_size = 5
num_features = 10
X = torch.rand(size=(batch_size,num_features))
mean = torch.mean(X,dim=0) # mean along every feature and therefore there will be 10 different means
std = torch.std(X,dim=0)
X = (X-mean)/std

print(f"Normalised Data :{X}")

Normalised Data :tensor([[-1.4327e+00,  1.5524e-01, -6.3545e-01,  2.5937e-01,  9.4289e-01,
         -6.0890e-01,  3.8253e-01,  1.1252e+00,  4.2250e-01,  7.5454e-04],
        [-5.5106e-01,  1.5899e+00,  8.7424e-01, -1.0110e+00,  1.1499e+00,
          1.5305e+00,  1.2008e+00,  4.8330e-01, -6.6470e-01, -1.0054e+00],
        [ 1.0955e+00, -5.7064e-01, -1.3766e+00, -8.7428e-01, -2.5748e-01,
         -1.1187e+00, -1.4135e+00, -1.4218e+00, -1.2974e+00,  3.1168e-01],
        [ 2.8755e-01, -1.0524e+00,  1.8848e-01,  1.4550e+00, -7.7960e-01,
          1.8778e-01,  3.6193e-01, -5.7360e-01,  1.2754e+00, -8.0501e-01],
        [ 6.0072e-01, -1.2217e-01,  9.4936e-01,  1.7089e-01, -1.0557e+00,
          9.2866e-03, -5.3169e-01,  3.8696e-01,  2.6426e-01,  1.4980e+00]])


In [ ]:
# Given two tensors A and B of shapes (N, D) and (M, D),
# compute the pairwise Euclidean distance matrix of shape (N, M)
# using broadcasting (no loops).
#
N = 10
D = 20
M = 30

A = torch.rand(size=(N,D))
B = torch.rand(size=(M,D))
DIST = torch.zeros(size=(N,M))

DIST = 


RuntimeError: The size of tensor a (10) must match the size of tensor b (30) at non-singleton dimension 0

In [57]:
b = torch.rand((3,)).to(device)
X = torch.rand((32,3,28,28)).to(device)
b = b.unsqueeze(1).unsqueeze(1)
print(b.shape)

# c = torch.broadcast_shapes(b.shape,X.shape)
y = X+b
print(y.shape)


torch.Size([3, 1, 1])
torch.Size([32, 3, 28, 28])


In [ ]:
# Given an input tensor x of shape (batch, channels, height, width)
# and a bias tensor b of shape (channels,),
# add the bias to each channel using broadcasting.

X = torch.rand((32,3,28,28)).to(device)

# One method of doing this:
'''
b = torch.rand((3,)).to(device)

b = b.unsqueeze(1)
b = b.unsqueeze(2).unsqueeze(2)
'''

# Other method of doing this:
'''
b = b.unsqueeze(1).unsqueeze(1)
print(b.shape)
'''

y = X+b
print(X.shape)
print(b.shape)
print(y.shape)



torch.Size([32, 3, 28, 28])
torch.Size([1, 3, 1, 1])
torch.Size([32, 3, 28, 28])


In [66]:
# Given vectors a (shape (3,)) and b (shape (4,)), 
# compute their outer product (3x4) using broadcasting.
a = torch.rand((4,))
b = torch.rand((3,))
# dot_ = b.view((3,1)) @ a.view((1,4))
dot_ = b.unsqueeze(1) @ a.unsqueeze(0)
print(dot_.shape)
print(dot_)

torch.Size([3, 4])
tensor([[0.3094, 0.4802, 0.9009, 0.3040],
        [0.0845, 0.1312, 0.2462, 0.0831],
        [0.0182, 0.0283, 0.0530, 0.0179]])


In [43]:
# Implement a function that normalizes tensor x 
# such that softmax is applied over the last dimension:
# def softmax(x):
#     exp_x = torch.exp(x - x.max(dim=-1, keepdim=True).values)
#     return exp_x / exp_x.sum(dim=-1, keepdim=True)
#
# Understand how broadcasting ensures correct denominator division.
X = torch.rand((32,3,28,28)).to(device)

In [ ]:
# Given A, B of shape (batch, dim), compute cosine similarity per batch:
# cosine_sim = (A * B).sum(dim=1) / (A.norm(dim=1) * B.norm(dim=1))
# Identify all broadcasted operations.
A = torch.rand((32,3))
B = torch.rand_like(A)
normA = torch.linalg.norm(A).item()
normB = torch.linalg.norm(B).item()
cos_sim2 = (A*B).sum(dim=1)/(normA*normB)
# assert(torch.sum(cos_sim1) == torch.sum(cos_sim2))

cos_sim1 = torch.nn.functional.cosine_similarity(A,B,dim=1)

print(cos_sim1)
print(cos_sim2)

tensor([0.6435, 0.9277, 0.6211, 0.8914, 0.8293, 0.8587, 0.9386, 0.9464, 0.8193,
        0.8682, 0.9238, 0.6506, 0.9756, 0.5769, 0.9193, 0.9966, 0.9461, 0.9666,
        0.7352, 0.9741, 0.7115, 0.9866, 0.9178, 0.8969, 0.9594, 0.7270, 0.8818,
        0.8078, 0.9874, 0.6902, 0.1950, 0.9810])
tensor([0.0121, 0.0310, 0.0182, 0.0328, 0.0278, 0.0284, 0.0178, 0.0291, 0.0194,
        0.0222, 0.0403, 0.0105, 0.0486, 0.0083, 0.0217, 0.0225, 0.0311, 0.0124,
        0.0236, 0.0092, 0.0269, 0.0537, 0.0180, 0.0179, 0.0423, 0.0312, 0.0330,
        0.0280, 0.0459, 0.0217, 0.0027, 0.0414])


In [ ]:
# Given an image batch x of shape (N, C, H, W)
# Normalize each channel using channel-wise mean and std (both of shape (C,))
# Use broadcasting to implement this.


In [47]:
# Given logits of shape (batch, num_classes)
# and labels as class indices of shape (batch,)
# implement one-hot encoding manually and compute MSE loss using broadcasting.
logits = torch.rand(size=(32,2))
labels = torch.randint(0,2, size=(32,))
logits[:,0], logits[:,1] = 1,0
labels = labels.unsqueeze(dim=1)
mse = (labels-logits)**2
print(logits)
print(labels)
print(mse.sum().item())

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])
tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0]])
32.0


In [18]:
x = torch.rand(5,4,3)
y = torch.rand(4,1)
# x + y  # Throws error.
# Fix it using view/unsqueeze so broadcasting works.
tensors = torch.broadcast_tensors(x,y)
z_ = tensors[0]+tensors[1]
y = y.unsqueeze(0)
y = y.expand(x.shape)

z = x+y

print(z_== z)

tensor([[[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]],

        [[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]],

        [[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]],

        [[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]],

        [[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]]])


In [ ]:
# Create a tensor image (1, 1, 5, 5) and kernel (1, 1, 3, 3)
# Use unfold() + broadcasting to manually perform 2D convolution
# (no nn.Conv2d, pure tensor ops).


In [15]:
a = torch.rand((2,3,4))
b = torch.rand((3,4))
print(torch.broadcast_shapes(a.shape, b.shape))

torch.Size([2, 3, 4])


In [23]:
# Implement your own function broadcast_add(a, b) that adds two tensors manually by:
# Expanding singleton dimensions yourself
# Using .expand() or .repeat()
# Without using a + b directly.

def add_(a:torch.TensorType, b:torch.TensorType)->torch.TensorType:
    
    req_shape = None
    res = is_broadcastable(a,b)
    if res[0] :
        req_shape = torch.broadcast_shapes(a.shape, b.shape)
        
    shape_a, shape_b = res[1],res[2]
    assert(len(shape_a) == len(shape_b))
    a = a.expand(req_shape)
    b = b.expand(req_shape)
    
    return a+b

In [24]:
add_(a,b)

Shape of A : torch.Size([2, 3, 4]) 
Shape of B : torch.Size([1, 3, 4])


tensor([[[0.5818, 0.8960, 0.5725, 0.2958],
         [0.6426, 0.7939, 1.9210, 0.2752],
         [1.2137, 0.3252, 1.0868, 0.2231]],

        [[1.1449, 1.0830, 0.9835, 0.9078],
         [1.2823, 0.4320, 1.7079, 0.2106],
         [1.0938, 0.5041, 0.8349, 0.9412]]])